Дано 2 диска радиусом σ. Задайте случайное положение этих дисков так, чтобы они не
перекрывались: $|r_{ij} |$ > 2σ, а также задайте случайные скорости с выполнением условия
($r_{ij} \cdot v_{ij}$ ) < 0. Найдите время до столкновения τcoll и смоделируйте их продвижение за это
время. Если время τcoll конечное, проверьте, что они действительно соприкоснутся. Если
τcoll = ∞, проверьте условие, ($r_{ij} \cdot v_{ij}$ ) = 0. Запустите эту тестовую программу хотя бы $10^7$
раз

In [45]:
import random, math

### Инициализируем начальние координаты точек. 
Условие неналожения: $|\Delta \textbf{r}_{ij}|>\sigma$.

In [8]:
N = 2 
sigma = 0.2
L = 1
condition = False
while condition == False:
    coor = [(random.uniform(sigma, L - sigma), random.uniform(sigma, L - sigma))]
    for k in range(1, N):
        a = (random.uniform(sigma, L - sigma), random.uniform(sigma, L - sigma))
        min_dist = min(math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) for b in coor) 
        if min_dist < 2.0 * sigma: 
            condition = False
            break
        else:
            coor.append(a)
            condition = True
print (coor)

[(0.5034021569695188, 0.6889162048645721), (0.37485788522788976, 0.213483180022337)]


### Задаем начальные скорости
If either Δv ⋅ Δr ≥ 0 or d < 0, then the quadratic equation has no solution for Δt > 0; otherwise we are guaranteed that Δt ≥ 0.
Where $d = (\Delta \textbf{v}_{ij}\cdot\Delta \textbf{r}_{ij})^{2}-\Delta \textbf{v}_{ij}^{2}(\Delta \textbf{r}_{ij}^{2}-\sigma^{2})$



In [29]:
scals = []
condition = False
while condition == False:
    vel = [(random.uniform(-1, 1), random.uniform(-1, 1))]
    for k in range(1, N):
        v = (random.uniform(-1, 1), random.uniform(-1, 1))
        del_r = (coor[k-1][0] - coor[k][0], coor[k-1][1] - coor[k][1])
        del_v = (v[0] - vel[0][0], v[1] - vel[0][1])
        scal = (del_r[0]*del_v[0] + del_r[1]*del_v[1])
        d = scal**2 + (del_v[0]**2 + del_v[1]**2)*(del_r[0]**2 + del_r[1]**2 - 4*sigma**2) 
        if scal >= 0 and d < 0:
            condition = False
            break
        else:
            vel.append(v)
            scals.append(scal)
            condition = True
            
print(vel)
print(scals)      

[(-0.7299636502198317, -0.2542842192389485), (0.6293921379986323, 0.4490135327312781)]
[0.5091083774182594]


### Время столкновения со стенками
Столкновение за момент времени $\Delta t$ происходит, если в этот момент одна из проекций координаты равна $\sigma$ (соответствующая проекция скороти отрицательна) или $L-\sigma$ (соответствющая проекция скорости положительна).

In [30]:
def wall_time(pos_a, vel_a, sigma):
    if vel_a > 0.0:
        del_t = (L - sigma - pos_a) / vel_a
    elif vel_a < 0.0:
        del_t = (pos_a - sigma) / abs(vel_a)
    else:
        del_t = float('inf')
    return del_t

wall_time(coor[0][0], vel[0][0], sigma)

0.4156400895827456

### Время столкновения между частицами
Столкновение частиц за момент времени $\Delta t$ происходит, если в этот момент расстояние между ними равняется их удвоенному радиусу. Т.е. должно выполняться уравнение $4 \sigma^{2} = \Delta \textbf{r'}_{ij}^{2}$, где $\Delta \textbf{r'}_{ij}$ - расстояние между парой частиц. Ранее мы уже генерировали скорости так, чтобы обеспечить столкновение частиц.

In [35]:
def pair_time(pos_a, vel_a, pos_b, vel_b, sigma):
    del_x = [pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]]
    del_x_sq = del_x[0] ** 2 + del_x[1] ** 2
    del_v = [vel_b[0] - vel_a[0], vel_b[1] - vel_a[1]]
    del_v_sq = del_v[0] ** 2 + del_v[1] ** 2
    scal = del_v[0] * del_x[0] + del_v[1] * del_x[1]
    d = scal ** 2 - del_v_sq * (del_x_sq - 4.0 * sigma ** 2)
    del_t = - (scal + math.sqrt(d)) / del_v_sq
    return del_t

t = pair_time(coor[0], vel[0], coor[1], vel[1], sigma)

### Продвижение за время $t_{coll}$
Считается, что в интервале между столкновениями скорости постоянны и на частиц не действуют другие силы.

In [42]:
def nc(pos_a, vel_a, pos_b, vel_b, t):
    nc_a = (pos_a[0] + vel_a[0]*t, pos_a[1] + vel_a[1]*t)
    nc_b = (pos_b[0] + vel_b[0]*t, pos_b[1] + vel_b[1]*t)
    new_coor = [nc_a, nc_b]
    return new_coor
new_coor = nc(coor[0], vel[0], coor[1], vel[1], t)   
del_r_sq = (new_coor[0][0] - new_coor[1][0])**2 + (new_coor[0][1] - new_coor[1][1])**2

In [43]:
print(4.0*sigma**2, del_r_sq) #проверка выполнения условия столкновения 

0.16000000000000003 0.16000000000000003
